# Read RADMC-3D density and *cartesianize*

In [ ]:
import numpy as np
from matplotlib.colors import LogNorm
from scipy.interpolate import interpn

from radmc3dPy.analyze import readData
import astropy.constants as c

au = c.au.cgs.value

In [ ]:
d = readData(ddens=True, binary=False)

r = d.grid.x
ri = d.grid.xi
th = d.grid.y
thi = d.grid.yi
ph = d.grid.z
phi = d.grid.zi
rho = d.rhodust[..., 0]

# Normalize with respect to mid-plane density

rho_mid = rho.max(-1).max(-1)
rho = rho / rho_mid[:, None, None]

inspect data: make a 2D vertical slice

In [ ]:
rri, tti = np.meshgrid(ri, thi, indexing='ij')

xxi = rri * np.sin(tti)
zzi = rri * np.cos(tti)

i_phi = 0

f, ax = plt.subplots(dpi=200)
vmax = 10.**np.ceil(np.log10(rho.max()))
cc =ax.pcolormesh(xxi / au, zzi / au, rho[:, :, i_phi], norm=LogNorm(vmin=1e-8 * vmax, vmax=vmax), shading='flat')
ax.set_aspect('equal')
pos = ax.get_position()
cax = f.add_axes([pos.x1, pos.y0, pos.height / 30, pos.height])
cb = plt.colorbar(cc, cax=cax)
cb.set_label('$\\rho_{dust}$')

## Prepare for interpolation

tricky part: we define the density only at grid centers, so there is a gap in $\phi$ direction between the last and the first grid center as the interpolation will not know about the periodic dimension. We fix this by shifting the first point to be at exactly $\phi=0$ and add a copy of this point at the end at $\phi=2 \pi$.

In [ ]:
ph_mod = np.hstack((ph - ph[0], 2 * np.pi))
rho_mod = np.concatenate((rho, rho[:, :, 0:1]), axis=2)

We create a cartesian grid, here `(x, y)` is in the plane while we call the height `z`.

In [ ]:
dx = 0.4

x = np.arange(-70, 70, dx) * au
y = np.arange(-70, 70, dx) * au
z = np.arange(-70, 70, dx) * au

X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

We translate the coordinates of that slice to spherical coordinates.

Note that phi goes from 0 to $2 \pi$ in the original data, but the output of `np.arctan2` has negative angles and needs to be shifted

In [ ]:
R = np.sqrt(X**2 + Y**2 + Z**2)
T = np.pi/2 - np.arctan2(Z, np.sqrt(X**2 + Y**2))
P = (np.arctan2(Y, X) + 2 * np.pi) % (2 * np.pi)

We create an array of new points, shape is (N, 3)

In [ ]:
points = np.array([R.ravel(), T.ravel(), P.ravel()]).T

We call the interpolation. There's values outside the box, so we assign them the value `0.0`.

In [ ]:
interp = interpn((r, th, ph_mod), rho_mod, points, fill_value=0.0, bounds_error=False)

The result is again one dimensional (one value per new point), so we need to reshape it to match the shape of the slice. After this, it is again 3-dimensional, since our slice has 3 dimensions, but the z-dimension is just one value here.

In [ ]:
interp = interp.reshape(X.shape)

Plot it

In [ ]:
f, axs = plt.subplots(1, 2, dpi=200, gridspec_kw={'width_ratios':[1, 1]}, figsize=(10, 5))

vmax = 10.**np.ceil(np.log10(interp.max()))

iy = len(y) // 2
iz = len(z) // 2

ax = axs[0]
cc1 = ax.pcolormesh(X[:, iy, :] / au, Z[:, iy, :] / au, interp[:, iy, :], norm=LogNorm(vmin=1e-6 * vmax, vmax=vmax), shading='auto', rasterized=True)

ax = axs[1]
cc2 = ax.pcolormesh(X[:, :, iz] / au, Y[:, :, iz] / au, interp[:, :, iz], norm=LogNorm(vmin=1e-6 * vmax, vmax=vmax), shading='auto', rasterized=True)


for ax, cc in zip(axs, [cc1, cc2]):
    ax.set_aspect('equal')
    pos = ax.get_position()
    cax = f.add_axes([pos.x0, pos.y1, pos.width, pos.height / 20])
    cb = plt.colorbar(cc, cax=cax, orientation='horizontal')
    cb.set_label('$\\rho$')
    cax.xaxis.set_label_position('top')
    cax.xaxis.set_ticks_position('top')
    ax.set_xlabel('x')
    
axs[0].set_ylabel('z')
axs[1].set_ylabel('y')
    
f.savefig('plot.pdf', transparent=True, bbox_inches='tight')

## Export data

store the data in a simple format

In [ ]:
np.savez('radmc3d_data.npz', x=x, y=y, z=z, rho=interp)

# Rendering

In [ ]:
from volrender import Renderer, TransferFunction, render_movie
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np

this can be loaded with:

In [ ]:
with np.load('radmc3d_data.npz') as f:
    x = f['x']
    y = f['y']
    z = f['z']
    rho = f['rho']

In [ ]:
vmax = rho.max()
datacube = LogNorm(vmin=vmax * 1e-8, vmax=vmax, clip=True)(rho.ravel()).reshape(rho.shape).data

In [ ]:
tf = TransferFunction(x0=[0.2, 0.5, 1.0], sigma=[0.05, 0.05, 0.01])
tf.colors[:, -1] = np.array([0.04, 0.02, 0.01])
r = Renderer(datacube, diagnostics=True, tf=tf)

In [ ]:
#r.render(phi=10, theta=-60)
r.render(phi=10, theta=60, bg=1.0)
f, ax = r.plot(diagnostics=True)
#f.savefig('render.pdf', transparent=True, bbox_inches='tight')

In [ ]:
nf = 96
render_movie(datacube, 60 * np.ones(nf), np.linspace(0, 360, nf + 1)[::-1], ncpu=4, tf=tf, bg=1.0)

In [ ]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls>
  <source src="movie.mp4" type="video/mp4">
</video>
""")